## 데이터 불러오기

In [ ]:
from geoband.API import *
import os
from glob import glob
import json
import pandas as pd
import numpy as np
import folium
from folium import plugins
import geopandas as gpd

In [ ]:
df1 = pd.read_csv('1.오산시_주정차단속(2018~2020).csv')
df2 = json.load(open('2.오산시_어린이교통사고_격자.geojson', encoding = 'utf-8'))
df3 = json.load(open('3.오산시_차량등록현황_격자.geojson', encoding = 'utf-8'))
df4 = json.load(open('4.오산시_연령별_거주인구격자(총인구).geojson', encoding = 'utf-8'))
df5 = json.load(open('5.오산시_연령별_거주인구격자(유소년).geojson', encoding = 'utf-8'))
df6 = json.load(open('6.오산시_연령별_거주인구격자(생산가능인구).geojson', encoding = 'utf-8'))
df7 = json.load(open('7.오산시_연령별_거주인구격자(고령).geojson', encoding = 'utf-8'))
df8 = pd.read_csv('8.오산시_유동인구(2019).csv')
df9 = pd.read_csv('9.오산시_어린이보호구역.csv')
df10 = pd.read_csv('10.오산시_학교위치정보.csv')
df11 = json.load(open('11.오산시_초등학교_통학구.geojson', encoding = 'utf-8'))
df12 = json.load(open('12.오산시_중학교_학군.geojson', encoding = 'utf-8'))
df13 = pd.read_csv('13.오산시_어린이집_유치원현황.csv')
df14 = pd.read_csv('14.오산시_기상데이터(2010~2019).csv')
df15 = pd.read_csv('15.오산시_무인교통단속카메라.csv')
df16 = pd.read_csv('16.오산시_도로안전표지표준데이터.csv')
df17 = json.load(open('17.오산시_횡단보도.geojson', encoding = 'utf-8'))
df18 = pd.read_csv('18.오산시_과속방지턱표준데이터.csv')
df19 = json.load(open('19.오산시_신호등.geojson', encoding = 'utf-8'))
df20 = pd.read_csv('20.오산시_CCTV설치현황.csv')
df21 = json.load(open('21.오산시_인도.geojson', encoding = 'utf-8'))
df22 = pd.read_csv('22.오산시_버스정류장.csv')
df23 = json.load(open('23.오산시_상세도로망_LV6.geojson', encoding = 'utf-8'))
df24 = pd.read_csv('24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
df25 = pd.read_csv('25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
df26 = pd.read_csv('26.평일_전일_오산시_혼잡시간강도_Level6.csv')
df27 = json.load(open('27.오산시_도로명주소_건물.geojson', encoding = 'utf-8'))
df28 = json.load(open('28.오산시_건물연면적_격자.geojson', encoding = 'utf-8'))
df29 = pd.read_csv('29.오산시_체육시설현황.csv')
df30 = pd.read_csv('30.오산시_학원_및_교습소_현황.csv')
df31 = json.load(open('31.오산시_법정경계(시군구).geojson', encoding = 'utf-8'))
df32 = json.load(open('32.오산시_행정경계(읍면동).geojson', encoding = 'utf-8'))
df33 = json.load(open('33.오산시_법정경계(읍면동).geojson', encoding = 'utf-8'))
df34 = json.load(open('34.오산시_지적도.geojson', encoding = 'utf-8'))

## 데이터 전처리

### 1. 격자(Polygon)형 데이터 전처리
- 100mx100m 격자의 중심점을 도출
- 중심점이 격자 내 좌표에 속해있다면, 격자에 속한 것으로 Count
- 활용한 Feature: 사고횟수, 차량개수, 유소년인구수, 생산가능인구수, 고령인구수, 건물연면적

### 2. 좌표형 데이터 전처리
- 100mx100m 격자의 위도의 최소/최대 및 경도의 최소/최대 범위를 구함
- 해당 격자내에 좌표가 속해있다면, 격자에 속한 것으로 Count
- 활용한 Feature: 주정차단속건수, 유동인구, 어린이보호구역개수, 어린이보호구역CCTV개수, 초등학교개수, 유치원/어린이집개수, 무인교통단속카메라개수, 도로안전표지판개수, 횡단보도개수, 과속방지턱개수, 신호등개수, 버스정류장개수, 건물개수, 체육시설개수, 학원교습소개수

### 3. MultiLineString형 데이터 전처리
- Buffer를 활용해 Polygon형태로 변환
- 격자의 Polygon과 Buffer의 Polygon이 겹치는(intersection)부분이 있다면 해당 격자에 속한 것으로 Count
- 활용한 Feature: 전체 추정교통량, 혼잡빈도강도, 혼잡시간강도

## 1. 격자(Polygon)형 데이터 전처리

### df2: 사고횟수

In [ ]:
# 2. 오산시 어린이 교통사고 격자

df2_1 = pd.DataFrame(columns=['격자명','사고횟수'])
for i in range(len(df2['features'])):
    a = df2['features'][i]['properties']['gid']
    b = df2['features'][i]['properties']['accident_cnt']
    c = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '사고횟수':b, '경도': c,'위도': d}
    df2_1 = df2_1.append(new_data, ignore_index=True)

df2_1['사고횟수'] = df2_1['사고횟수'].astype('int32')
df2_1

### df3:  차량개수

In [ ]:
# 3. 오산시 차량등록현황 격자

df3_1 = pd.DataFrame(columns=['격자명','차량개수'])
for i in range(len(df3['features'])):
    a = df3['features'][i]['properties']['gid']
    b = df3['features'][i]['properties']['car_cnt']
    c = np.mean(np.array(df3['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df3['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '차량개수':b, '경도': c,'위도': d}
    df3_1 = df3_1.append(new_data, ignore_index=True)

df3_1['차량개수'] = df3_1['차량개수'].astype('int32')
df3_1

### df5: 유소년인구수

In [ ]:
# 5. 오산시 연령별 거주인구 격자(유소년)

df5_1 = pd.DataFrame(columns=['격자명','유소년인구수'])
for i in range(len(df5['features'])):
    a = df5['features'][i]['properties']['gid']
    b = df5['features'][i]['properties']['val']
    c = np.mean(np.array(df5['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df5['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '유소년인구수':b, '경도': c,'위도': d}
    df5_1 = df5_1.append(new_data, ignore_index=True)

df5_1['유소년인구수'] = df5_1['유소년인구수'].astype('float64')
df5_1 = df5_1.fillna(0)
df5_1

### df6: 생산가능인구수

In [ ]:
# 6. 오산시 연령별 거주인구 격자(생산가능인구)

df6_1 = pd.DataFrame(columns=['격자명','생산가능인구수'])
for i in range(len(df6['features'])):
    a = df6['features'][i]['properties']['gid']
    b = df6['features'][i]['properties']['val']
    c = np.mean(np.array(df6['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df6['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '생산가능인구수':b, '경도': c,'위도': d}
    df6_1 = df6_1.append(new_data, ignore_index=True)

df6_1['생산가능인구수'] = df6_1['생산가능인구수'].astype('float64')
df6_1 = df6_1.fillna(0)
df6_1

### df7: 고령인구수

In [ ]:
# 7. 오산시 연령별 거주인구 격자(고령)

df7_1 = pd.DataFrame(columns=['격자명','고령인구수'])
for i in range(len(df7['features'])):
    a = df7['features'][i]['properties']['gid']
    b = df7['features'][i]['properties']['val']
    c = np.mean(np.array(df7['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df7['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '고령인구수':b, '경도': c,'위도': d}
    df7_1 = df7_1.append(new_data, ignore_index=True)

df7_1['고령인구수'] = df7_1['고령인구수'].astype('float64')
df7_1 = df7_1.fillna(0)
df7_1

### df28: 건물연면적

In [ ]:
# 28. 오산시 건물 연면적 격자

df28_1 = pd.DataFrame(columns=['격자명','건물연면적'])
for i in range(len(df28['features'])):
    a = df28['features'][i]['properties']['gid']
    b = df28['features'][i]['properties']['val']
    c = np.mean(np.array(df28['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df28['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '건물연면적':b, '경도': c,'위도': d}
    df28_1 = df28_1.append(new_data, ignore_index=True)

df28_1['건물연면적'] = df28_1['건물연면적'].astype('float64')
df28_1 = df28_1.fillna(0)
df28_1

## 2. 좌표형 데이터 전처리

### df1: 주정차단속건수

In [ ]:
# 1. 오산시_주정차단속(2018~2020)

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df1_1 = pd.DataFrame({'격자명':list_tmp})
df1_1['주정차단속건수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df1)):
        if (df1.loc[j, '단속위치_위도'] >= min_lat) & (df1.loc[j, '단속위치_위도'] <= max_lat) & (df1.loc[j, '단속위치_경도'] >= min_lon) & (df1.loc[j, '단속위치_경도'] <= max_lon):
            df1_1.loc[i,'주정차단속건수'] = df1_1.loc[i,'주정차단속건수']+1

df1_1

### df8: 유동인구

In [ ]:
# 어린이가 많이 다니는 14시 ~ 20시로 한정
df8_1 = df8.loc[:,['lon','lat','TMST_14','TMST_15','TMST_16','TMST_17','TMST_18','TMST_19','TMST_20']]
df8_1['mean'] = (df8_1['TMST_14'] + df8_1['TMST_15'] + df8_1['TMST_16'] + df8_1['TMST_17'] + df8_1['TMST_18'] + df8_1['TMST_19'] +df8_1['TMST_20'])/7
df8_1 = df8_1.loc[:,['lon','lat','mean']]

# 위도와 경도기준 groupby
df8_2 = df8_1.groupby(['lon','lat']).sum()
df8_2 = df8_2.reset_index()
df8_2 = df8_2.reindex(columns=['lat','lon','mean'])

# 유동인구를 격자에 할당
list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df8_1 = pd.DataFrame({'격자명':list_tmp})
df8_1['유동인구'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    if i % 100 == 0:
        print('Process: ',i,'단계')
    
    for j in range(len(df8_2)):
        if (df8_2.loc[j, 'lat'] >= min_lat) & (df8_2.loc[j, 'lat'] <= max_lat) & (df8_2.loc[j, 'lon'] >= min_lon) & (df8_2.loc[j, 'lon'] <= max_lon):
            df8_1.loc[i,'유동인구'] = df8_1.loc[i,'유동인구'] + df8_2.loc[j,'mean']
df8_1

### df9: 어린이보호구역개수

In [ ]:
# 9. 오산시_어린이보호구역

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df9_1 = pd.DataFrame({'격자명':list_tmp})
df9_1['어린이보호구역개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df9)):
        if (df9.loc[j, '보호구역_위도'] >= min_lat) & (df9.loc[j, '보호구역_위도'] <= max_lat) & (df9.loc[j, '보호구역_경도'] >= min_lon) & (df9.loc[j, '보호구역_경도'] <= max_lon):
            df9_1.loc[i,'어린이보호구역개수'] = df9_1.loc[i,'어린이보호구역개수']+1
df9_1

### df9: 오산시 어린이보호구역CCTV개수

In [ ]:
# 9. 오산시_어린이보호구역

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df9_2 = pd.DataFrame({'격자명':list_tmp})
df9_2['어린이보호구역CCTV개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df9)):
        if (df9.loc[j, '보호구역_위도'] >= min_lat) & (df9.loc[j, '보호구역_위도'] <= max_lat) & (df9.loc[j, '보호구역_경도'] >= min_lon) & (df9.loc[j, '보호구역_경도'] <= max_lon):
            df9_2.loc[i,'어린이보호구역CCTV개수'] = df9_2.loc[i,'어린이보호구역CCTV개수']+df9.loc[j, 'CCTV설치대수']
df9_2

### df10: 초등학교개수

In [ ]:
# 10. 오산시_학교위치정보

df10 = df10[df10['학교구분'] == '초등학교'].reset_index() # 초등학교 개수만 필터링

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df10_1 = pd.DataFrame({'격자명':list_tmp})
df10_1['초등학교개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df10)):
        if (df10.loc[j, '학교위치_위도'] >= min_lat) & (df10.loc[j, '학교위치_위도'] <= max_lat) & (df10.loc[j, '학교위치_경도'] >= min_lon) & (df10.loc[j, '학교위치_경도'] <= max_lon):
            df10_1.loc[i,'초등학교개수'] = df10_1.loc[i,'초등학교개수']+1
df10_1

### df13: 유치원/어린이집개수

In [ ]:
# 13. 오산시_어린이집_유치원현황

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df13_1 = pd.DataFrame({'격자명':list_tmp})
df13_1['유치원/어린이집개수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df13)):
        if (df13.loc[j, '시설위치_위도'] >= min_lat) & (df13.loc[j, '시설위치_위도'] <= max_lat) & (df13.loc[j, '시설위치_경도'] >= min_lon) & (df13.loc[j, '시설위치_경도'] <= max_lon):
            df13_1.loc[i,'유치원/어린이집개수'] = df13_1.loc[i,'유치원/어린이집개수']+1
df13_1

### df15: 무인교통단속카메라개수

In [ ]:
# 15. 오산시_무인교통단속카메라

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df15_1 = pd.DataFrame({'격자명':list_tmp})
df15_1['무인교통단속카메라개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df15)):
        if (df15.loc[j, '설치위치_위도'] >= min_lat) & (df15.loc[j, '설치위치_위도'] <= max_lat) & (df15.loc[j, '설치위치_경도'] >= min_lon) & (df15.loc[j, '설치위치_경도'] <= max_lon):
            df15_1.loc[i,'무인교통단속카메라개수'] = df15_1.loc[i,'무인교통단속카메라개수']+1
df15_1

### df16: 도로안전표지판개수

In [ ]:
# 16. 오산시_도로안전표지표준데이터

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)

df16_1 = pd.DataFrame({'격자명':list_tmp})
df16_1['도로안전표지판개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df16)):
        if (df16.loc[j, 'lat'] >= min_lat) & (df16.loc[j, 'lat'] <= max_lat) & (df16.loc[j, 'lon'] >= min_lon) & (df16.loc[j, 'lon'] <= max_lon):
            df16_1.loc[i,'도로안전표지판개수'] = df16_1.loc[i,'도로안전표지판개수']+1
df16_1

### df17: 횡단보도개수

In [ ]:
# 17. 오산시_횡단보도

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df17_1 = pd.DataFrame({'격자명':list_tmp})
df17_1['횡단보도개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df17['features'])):
        for k in range(len(df17['features'][j]['geometry']['coordinates'][0][0])):
            lat = df17['features'][j]['geometry']['coordinates'][0][0][k][1]
            lon = df17['features'][j]['geometry']['coordinates'][0][0][k][0]
        
        
            if (lat >= min_lat) & (lat <= max_lat) & (lon >= min_lon) & (lon <= max_lon):
                df17_1.loc[i,'횡단보도개수'] = df17_1.loc[i,'횡단보도개수']+1
                break
df17_1

### df18: 과속방지턱개수

In [ ]:
# 18. 오산시_과속방지턱표준데이터

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df18_1 = pd.DataFrame({'격자명':list_tmp})
df18_1['과속방지턱개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df18)):
        if (df18.loc[j, '설치위치_위도'] >= min_lat) & (df18.loc[j, '설치위치_위도'] <= max_lat) & (df18.loc[j, '설치위치_경도'] >= min_lon) & (df18.loc[j, '설치위치_경도'] <= max_lon):
            df18_1.loc[i,'과속방지턱개수'] = df18_1.loc[i,'과속방지턱개수']+1
df18_1

### df19: 신호등개수

In [ ]:
# 19. 오산시_신호등

# geojson내 coordinate를 dataframe화
list_tmp = []

for i in range(len(df19['features'])):
    tmp = df19['features'][i]['geometry']['coordinates']
    list_tmp.extend(tmp)
    
df19_tmp = pd.DataFrame(columns=['설치위치_위도','설치위치_경도'])

for i in range(0, 2*len(df19['features']), 2):
    a = list_tmp[i]
    b = list_tmp[i+1]
    tmp = {'설치위치_위도':b, '설치위치_경도':a}
    df19_tmp = df19_tmp.append(tmp, ignore_index=True)

# 격자에 할당
list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df19_1 = pd.DataFrame({'격자명':list_tmp})
df19_1['신호등개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df19_tmp)):
        if (df19_tmp.loc[j, '설치위치_위도'] >= min_lat) & (df19_tmp.loc[j, '설치위치_위도'] <= max_lat) & (df19_tmp.loc[j, '설치위치_경도'] >= min_lon) & (df19_tmp.loc[j, '설치위치_경도'] <= max_lon):
            df19_1.loc[i,'신호등개수'] = df19_1.loc[i,'신호등개수']+1
df19_1

### df22: 버스정류장개수

In [ ]:
# 22. 오산시_버스정류장

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df22_1 = pd.DataFrame({'격자명':list_tmp})
df22_1['버스정류장개수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df22)):
        if (df22.loc[j, '정류장 위치_위도'] >= min_lat) & (df22.loc[j, '정류장 위치_위도'] <= max_lat) & (df22.loc[j, '정류장 위치_경도'] >= min_lon) & (df22.loc[j, '정류장 위치_경도'] <= max_lon):
            df22_1.loc[i,'버스정류장개수'] = df22_1.loc[i,'버스정류장개수']+1
df22_1

### df27: 건물개수

In [ ]:
# 27. 오산시_도로명주소_건물

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df27_1 = pd.DataFrame({'격자명':list_tmp})
df27_1['건물개수'] = 0


for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0] 
    
    for j in range(len(df27['features'])):
        for k in range(len(df27['features'][j]['geometry']['coordinates'][0][0])):
            lat = df27['features'][j]['geometry']['coordinates'][0][0][k][1]
            lon = df27['features'][j]['geometry']['coordinates'][0][0][k][0]
        
        
            if (lat >= min_lat) & (lat <= max_lat) & (lon >= min_lon) & (lon <= max_lon):
                df27_1.loc[i,'건물개수'] = df27_1.loc[i,'건물개수']+1
                break
df27_1

### df29: 체육시설개수

In [ ]:
# 29. 오산시_체육시설현황

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df29_1 = pd.DataFrame({'격자명':list_tmp})
df29_1['체육시설개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df29)):
        if (df29.loc[j, '설치위치_위도'] >= min_lat) & (df29.loc[j, '설치위치_위도'] <= max_lat) & (df29.loc[j, '설치위치_경도'] >= min_lon) & (df29.loc[j, '설치위치_경도'] <= max_lon):
            df29_1.loc[i,'체육시설개수'] = df29_1.loc[i,'체육시설개수']+1
df29_1

### df30: 학원교습소개수

In [ ]:
# 30. 오산시_학원_및_교습소_현황

list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df30_1 = pd.DataFrame({'격자명':list_tmp})
df30_1['학원교습소개수'] = 0

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df30)):
        if (df30.loc[j, '시설위치_위도'] >= min_lat) & (df30.loc[j, '시설위치_위도'] <= max_lat) & (df30.loc[j, '시설위치_경도'] >= min_lon) & (df30.loc[j, '시설위치_경도'] <= max_lon):
            df30_1.loc[i,'학원교습소개수'] = df30_1.loc[i,'학원교습소개수']+1
df30_1

### 3. MultiLineString형 데이터 전처리

### MultiLineString을 Buffer를 활용하여 Polygon형태로 변환

In [ ]:
# 100m x 100m 격자 불러오기
df_100m = gpd.read_file('4.오산시_연령별_거주인구격자(총인구).geojson')
df_100m = df_100m.drop('val',axis=1) # 여기서 경도, 위도, geometry만 가져온다
 
# geopandas를 활용하여 상세도로망 데이터를 geojson --> csv
df23_1 = gpd.read_file('23.오산시_상세도로망_LV6.geojson')

# multilinestring --> buffer 변환
for i in range(len(df23_1)):
    df23_1.loc[i,'geometry'] = df23_1.loc[i,'geometry'].buffer(0.00005)

df23_1

### df24: 전체 추정교통량

In [ ]:
# 시간적 범위는 어린이가 가장 많이 몰리는 시간대인 14~20시로 한정
df24_1 = df24[df24['시간적범위'].isin(['15','16','17','18','19','20'])] 

# 상세도로망별 평균 "전체 추정교통량"도출
df24_2 = df24_1.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_2 = df24_2.reset_index()

# df23: 상세도로망 데이터와 연관짓기 위해 뒤에 상세도로망_LinkID2자리 삭제
for i in range(len(df24_2)):
    df24_2.loc[i,'상세도로망_LinkID'] = str(df24_2.iloc[i,0])[:-2]

# 뒤에 2자리를 지운 후 다시 groupby 실시
df24_3 = df24_2.groupby('상세도로망_LinkID').mean()[['전체 추정교통량']]
df24_3 = df24_3.reset_index()

# df23과 df24의 link_id의 열이름 통일
df24_3 = df24_3.rename(columns={'상세도로망_LinkID':'link_id'})

# link_id를 활용해 두 Dataframe을 merge
df_traffic = pd.merge(df23_1[['link_id','geometry']], df24_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()

# 격자별 "전체 추정교통량"을 0으로 초기화
df_100m['전체 추정교통량'] = 0

# 인덱스 초기화
df_traffic = df_traffic.reset_index(drop=True)

# 격자에 할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_100m[df_100m['geometry'].intersection(buffer).is_empty == False]['전체 추정교통량'].index):
        df_100m.loc[i,'전체 추정교통량'] += df_traffic.loc[k,'전체 추정교통량']

### df25: 혼잡빈도강도

In [ ]:
# 격자별 혼잡빈도강도 초기화
df_100m['혼잡빈도강도'] = 0

# 필요한 column만 불러온 후 groupby
df25_1 = df25[['상세도로망_LinkID','혼잡빈도강도']]

df25_2 = df25_1.groupby('상세도로망_LinkID').mean()[['혼잡빈도강도']]
df25_2 = df25_2.reset_index()

# link_ID 끝 두자리 제거
for i in range(len(df25_2)):
    df25_2.loc[i,'상세도로망_LinkID'] = str(df25_2.iloc[i,0])[:-2]
    
# groupby 및 merge
df25_3 = df25_2.groupby('상세도로망_LinkID').mean()[['혼잡빈도강도']]
df25_3 = df25_3.reset_index()
df25_3 = df25_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df25_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()
df_traffic = df_traffic.reset_index(drop=True)

# 격자에 할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_100m[df_100m['geometry'].intersection(buffer).is_empty == False]['혼잡빈도강도'].index):
        df_100m.loc[i,'혼잡빈도강도'] += df_traffic.loc[k,'혼잡빈도강도']

### df26: 혼잡시간강도

In [ ]:
# 격자별 혼잡빈도강도 초기화 및 필요한 Column만 구분
df_100m['혼잡시간강도'] = 0
df26_1 = df26[['상세도로망_LinkID','혼잡시간강도']]

# 필요데이터 groupby 및 merge
df26_2 = df26_1.groupby('상세도로망_LinkID').mean()[['혼잡시간강도']]
df26_2 = df26_2.reset_index()

for i in range(len(df26_2)):
    df26_2.loc[i,'상세도로망_LinkID'] = str(df26_2.iloc[i,0])[:-2]

df26_3 = df26_2.groupby('상세도로망_LinkID').mean()[['혼잡시간강도']]
df26_3 = df26_3.reset_index()
df26_3 = df26_3.rename(columns={'상세도로망_LinkID':'link_id'})

df_traffic = pd.merge(df23_1[['link_id','geometry']], df26_3, on='link_id', how='outer')
df_traffic = df_traffic.dropna()
df_traffic = df_traffic.reset_index(drop=True)

# 격자에 할당
for k in range(len(df_traffic)):
    buffer = df_traffic['geometry'][k].buffer(0.0001)
        
    for i in list(df_100m[df_100m['geometry'].intersection(buffer).is_empty == False]['혼잡시간강도'].index):
        df_100m.loc[i,'혼잡시간강도'] += df_traffic.loc[k,'혼잡시간강도']
        
df_100m

## Concat

In [ ]:
# 최종 데이터프레임 병합

df_final = pd.concat([df1_1.iloc[:,[0,1]],
                    df2_1.iloc[:,1],
                    df3_1.iloc[:,1],
                    df5_1.iloc[:,1],
                    df6_1.iloc[:,1],
                    df7_1.iloc[:,1],
                    df8_1.iloc[:,1],
                    df9_1.iloc[:,1],
                    df9_2.iloc[:,1],
                    df10_1.iloc[:,1],
                    df13_1.iloc[:,1],
                    df15_1.iloc[:,1],
                    df16_1.iloc[:,1],
                    df17_1.iloc[:,1],
                    df18_1.iloc[:,1],
                    df19_1.iloc[:,1],
                    df22_1.iloc[:,1],
                    df_100m.iloc[:,[2,3,4]],
                    df27_1.iloc[:,1],
                    df28_1.iloc[:,1],
                    df29_1.iloc[:,1],
                    df30_1.iloc[:,1]], axis=1)

df_final

# 모델링 절차

## 1. 세부 데이터 설정

어린이보호구역 외의 어린이 교통사고 위험지역 20개를 선정하기 위해, 이미 어린이보호 구역이 포함된 격자는 제외하고 다음의 후보군 격자와 모델링 격자를 구축하였다.

1. 어린이 교통사고 위험지역 후보군을 설정한다.  

    (다음의 조건 중 1개 이상 만족 시, 후보군)
    - 격자 내 초등학교가 1개 이상 
    - 격자 내 유치원 및 어린이집이 1개 이상
    - 격자 내 학원 및 교습소가 1개 이상


2. 모델을 통해 훈련시킬 격자 데이터는 기존 어린이보호구역과 후보군을 제외한다. 

## 2. Feature selection

1. Logistic Regression 모델을 통한 P-VALUE로 유의미하지 않은 변수제거 (a = 0.1)
 - p-value 0.1 이상의 변수를 제거한다.
2. 다중공선성을 띄는 변수를 분산 인플레이션 계수(VIF)를 통해 제거 
 - VIF Factor 7이상의 변수를 제거한다.

## 3. Modeling

3가지 모델을 수립 후, 성능이 우수한 모델로 설정한 후보군의 사고발생확률 도출
1. KNN Classfier
2. RandomForest Classifier
3. LightGbm

## 4. 후보군 격자들의 사고 발생 확률 예측

가장 성능이 우수한 모델을 통해 어린이 교통사고 위험지역 20개소를 예측하고, 확률 순으로 내림차순하여 시각화한다.

## 5. 어린이 교통사고 위험지역 20개소 선정

    - 상위 TOP 20 선정 순서

    1. 기존 어린이보호 구역의 시설명이 포함된 격자의 경우, 후보에서 제거한다.
    2. 다음의 사고 확률이 높은 새로운 후보를 등록한 후, 다시 기존 어린이보호 구역의 시설명이 포함되는지 비교한다. 
    3. 1과 2의 작업을 기존 어린이보호 구역의 시설명이 포함되지 않을 때까지 반복한다.


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
import lightgbm as lgb
import time
import random

### 1. 세부 데이터 설정

#### 유치원 및 어린이집 변수 추가하여 후보군 생성 - df_child

    (다음의 조건 중 1개 이상 만족 시, 후보군)
    - 격자 내 초등학교가 1개 이상 
    - 격자 내 유치원 및 어린이집이 1개 이상
    - 격자 내 학원 및 교습소가 1개 이상
앞서 할당시킨 4510개의 격자 데이터에, **어린이 대상**의 오산시_학원_및_교습소_현황 데이터를 추가한다.

In [ ]:
df30['교습과정명'].unique()

많은 학원 유형 중, 어린이 교통사고에 한정시키기 위해 **'초등', '저학년', '초1'...'초6'이 들어간 것**으로 선정<br>
이들 키워드 중 1개라도 속하면 초등학생 학원이라고 분류하였다.

In [ ]:
cond1 = df30['교습과정명'].str.contains('초등', na=False)
cond2 = df30['교습과정명'].str.contains('저학년', na=False)
cond3 = df30['교습과정명'].str.contains('초1', na=False)
cond4 = df30['교습과정명'].str.contains('초2', na=False)
cond5 = df30['교습과정명'].str.contains('초3', na=False)
cond6 = df30['교습과정명'].str.contains('초4', na=False)
cond7 = df30['교습과정명'].str.contains('초5', na=False)
cond8 = df30['교습과정명'].str.contains('초6', na=False)

df30_1 = df30.loc[cond1 | cond2 | cond3 | cond4 | cond5 | cond6 | cond7 | cond8,:]
df30_1 = df30_1.reset_index(drop=True)
df30_1

유치원생 및 초등학생 학원의 수를 격자화한다.

In [ ]:
# 유치원생 및 초등학생 학원의 수를 격자화1
list_tmp = []
for i in range(len(df28['features'])):
    tmp = df28['features'][i]['properties']['gid']
    list_tmp.append(tmp)


df30_2 = pd.DataFrame({'격자명':list_tmp})
df30_2['유치원및초등생학원개수'] = 0
df30_2

In [ ]:
# 유치원생 및 초등학생 학원의 수를 격자화2

for i in range(len(df28['features'])):
    arr = np.array(df28['features'][i]['geometry']['coordinates'][0][0])
    min_arr = np.min(arr, axis=0)
    max_arr = np.max(arr, axis=0)
    
    # 위도
    min_lat = min_arr[1]
    max_lat = max_arr[1]
    
    # 경도
    min_lon = min_arr[0]
    max_lon = max_arr[0]
    
    for j in range(len(df30_1)):
        if (df30_1.loc[j, '시설위치_위도'] >= min_lat) & (df30_1.loc[j, '시설위치_위도'] <= max_lat) & (df30_1.loc[j, '시설위치_경도'] >= min_lon) & (df30_1.loc[j, '시설위치_경도'] <= max_lon):
            df30_2.loc[i,'유치원및초등생학원개수'] = df30_2.loc[i,'유치원및초등생학원개수']+1
df30_2

기존 df_final 데이터와 학원/교습소 정보를 추가한 데이터프레임을 병합한다.

In [ ]:
df_final2 = pd.concat([df_final,df30_2.iloc[:,1]], axis=1)
df_final2

### 기존 어린이보호구역 제거

In [ ]:
# 기존 어린이 보호구역이 1개라도 있는 격자 제거
data = df_final2[df_final2['어린이보호구역개수'] >=1]
data  # 88개 격자 제거

In [ ]:
# 위 index를 제외한 index 
data_idx = df_final2.index.drop(list(data.index))

# 위에서 추출한 index를 df에 적용
df_final2 = df_final2.loc[data_idx,:]
df_final2 # 기존 어린이보호구역이 제거된 df_final2

### 후보군 도출 - df_child

In [ ]:
# 처음에 말했던 것 처럼 아래 원리에 따라 후보군을 지정해준다.
 
'''
- 후보군 구분
  (1) 초등학교, 유치원/어린이집, 학원이 1개라도 있는 격자만이 가능.
  (2) 추가적으로 어린이 보호구역이 1개라도 있는 격자는 후보 불가 (앞에서 제거함)
'''
# 초등학교, 유치원/어린이집, 학원이 1개라도 있는 격자만이 가능.
cond1 = df_final2['초등학교개수'] >= 1
cond2 = df_final2['유치원/어린이집개수'] >= 1
cond3 = df_final2['유치원및초등생학원개수'] >= 1
df_child = df_final2[cond1 | cond2 | cond3]
df_child

In [ ]:
df_child[['초등학교개수','유치원/어린이집개수','유치원및초등생학원개수']][cond1 | cond2 | cond3] # 조건만족하는지 확인

In [ ]:
# 위 253개의 index를 제외한 나머지가 바로 train의 index1 --> train의 index만 따로 추출
data2_idx = df_final2.index.drop(list(df_child.index))

# 위에서 추출한 index를 df에 적용
data_real = df_final2.loc[data2_idx,:]
data_real

In [ ]:
data_real.columns

모델링에 사용하지 않는 변수 삭제
- 유치원및 초등학생학원개수

### 253개 후보군 - df_child

In [ ]:
# 후보군에서의 어린이 보호구역 개수,유치원및초등생학원개수 column 삭제

df_child.drop(['유치원및초등생학원개수'],axis=1,inplace=True)
df_child.shape

### 4169개 모델링 대상 - df_real

In [ ]:
data_real.drop(['유치원및초등생학원개수'],axis=1,inplace=True)
data_real.shape

In [ ]:
data_real.reset_index(drop=True,inplace=True)
data_real.columns

종속변수인 사고횟수를 격자 내에서 1번이라도 일어나면 1, 그 외는 0으로 할당한다.

In [ ]:
for i in range(len(data_real)):
    if data_real.loc[i,'사고횟수'] >= 1:
        data_real.loc[i,'사고횟수'] = 1


#### 모델생성용: 4169개 격자, 위험지역 후보군: 253개 격자

In [ ]:
# 추후 격자명을 분석하기 위해 복사본 생성
data1_original = data_real.copy()
df_child_original = df_child.copy()
# 격자명 변수 삭제
data_real.drop('격자명',axis=1,inplace=True)
df_child.drop('격자명',axis=1,inplace=True)


In [ ]:
data_real.shape, df_child.shape

In [ ]:
# 사고횟수를 y로, 그 외 변수는 모두 x

feature_columns = list(data_real.columns)
feature_columns.remove('사고횟수')

data_x = data_real[feature_columns]
data_y = data_real[['사고횟수']]
df_child_x = df_child[feature_columns]
df_child_y = df_child[['사고횟수']]
data_x.shape, data_y.shape, df_child_x.shape, df_child_y.shape

#### 오버샘플링

종속변수가 편향되어 있으므로, **오버샘플링 기법**을 통해 종속변수의 개수 평준화를 한다.

우선, train set 과 test set을 나누고 모델링에 활용되는 train set에 대해서만 오버샘플링 기법을 적용한다.

In [ ]:
# 종속변수 분포 확인
data_y.value_counts()

#### Train / test split

모델링을 평가하기 위해 4169개의 데이터를 train set, test set으로 나눈다.

In [ ]:
# train vs test 나누기
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size = 0.2, random_state = 42, shuffle=True, stratify=data_y) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

In [ ]:
train = pd.concat([train_x,train_y],axis=1)
train_0 = train[train['사고횟수'] == 0]
train_1 = train[train['사고횟수'] == 1]

train_1_over = train_1.sample(3120, replace=True, random_state=10)
train_over = pd.concat([train_0, train_1_over], axis=0)
train_over.사고횟수.value_counts()

In [ ]:
# 독립변수와 종속변수 분류
feature_columns = list(train_over.columns.difference(['사고횟수','격자명']))
X = train_over[feature_columns]
y = train_over[['사고횟수']]

# 이름을 기존과 동일하게 변경
train_x = X.copy()
train_y = y.copy()
train_x.shape, train_y.shape, test_x.shape, test_y.shape

## 2. Feature selection

### 변수 간의 상관계수를 도출하여 유의미하지 않은 변수 제거

1. 유치원/어린이집개수
2. 어린이보호구역개수
3. 초등학교개수
4. 어린이보호구역CCTV개수

In [ ]:
train_x.corr()

In [ ]:
train_x = train_x.drop(['유치원/어린이집개수','어린이보호구역개수','초등학교개수','어린이보호구역CCTV개수'], axis=1)

### 설명변수 정규화

Logistic Regression을 진행하기 전 설명변수들의 각각의 변수값의 분포는 매우 다양하므로, 0과 1 사이의 값으로 정규화 진행한다.


In [ ]:
# train data 정규화
min_max_scaler = preprocessing.MinMaxScaler()
train_x2 = min_max_scaler.fit_transform(train_x)
train_x2 = pd.DataFrame(train_x2, columns = train_x.columns)
train_y.reset_index(drop=True,inplace=True)

### Logistic Regression을 통한 P-VALUE로 유의미하지 않은 변수제거 (a = 0.1)

In [ ]:
# 유의미하지 않은 변수를 선정하기위해 로지스틱 다중회귀분석을 통한 p-value를 활용 (기준값: a=0.1)
## 결론적으로 3개의 독립변수가 생존
train_x2 = add_constant(train_x2)
model = sm.Logit(train_y, train_x2)
fit_model = model.fit()
fit_model.summary(alpha=0.1)

### 분석 결과

Logistic Regression 결과, p-value가 0.1보다 작은 변수들은 해당 모델의 귀무가설(회귀계수 = 0) 을 기각하게 되므로 유의미한 변수이다.

따라서, p-value가 0.1보다 큰 변수들은 feature selection에서 제거한다.

제거된 변수는 다음과 같다.<br>
1. 과속방지턱개수 : p-value 0.134
2. 무인교통단속카메라개수 : p-value 0.877
3. 생산가능인구수 : p-value 0.493
4. 신호등개수 : p-value 0.327
5. 유소년인구수 : p-value 0.461
6. 주정차단속건수 : p-value 0.630
7. 차량개수 : p-value 0.837
8. 학원교습소개수 : p-value 0.799
9. 혼잡빈도강도 : p-value 0.518



### 독립변수 재설정

In [ ]:
feature_columns = ['건물개수','건물연면적','고령인구수','버스정류장개수','도로안전표지판개수','유동인구','전체 추정교통량','체육시설개수','혼잡시간강도','횡단보도개수']
len(feature_columns)

In [ ]:
train_x_final = train_x.loc[:,feature_columns]
train_y_final = train_y
test_x_final = test_x.loc[:,feature_columns]
test_y_final = test_y

df_child_x = df_child_x.loc[:,feature_columns] # 후보군 data도 똑같이 변환
print(train_x_final.shape, train_y_final.shape, test_x_final.shape, test_y_final.shape)

###  분산 인플레이션 계수(VIF) 계산

VIF factor 가 7이상인 다중공선성을 띄는 변수를 제거한다.<br>


In [ ]:
train_x_final = add_constant(train_x_final) 
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    train_x_final.values, i) for i in range(train_x_final.shape[1])]
vif["features"] = train_x_final.columns
vif

In [ ]:
# logistic regression 및 VIF에 들어갔던 상수항 제거
train_x_final = train_x_final.drop('const',axis=1)
train_x_final.shape, test_x_final.shape

#### 분석 결과

상관계수와 logistic regression 모델을 통해 제거되고 남은 변수들만의 다중공선성 효과가 미비하다고 할 수 있다.

따라서 **위의 10가지 변수에 대해 머신러닝 모델**을 수립하고자 한다.

모델링 과정은 다음과 같다.

## 3. Modeling

### 정규화

새로 정의한 10개의 변수에 대해 다시 정규화한다.

In [ ]:
# train data & test data 정규화
min_max_scaler = preprocessing.MinMaxScaler()
train_x_final2 = min_max_scaler.fit_transform(train_x_final)
test_x_final2 = min_max_scaler.fit_transform(test_x_final)
df_child_x = min_max_scaler.fit_transform(df_child_x)

#정규화 결과는 array 이므로 DataFrame화
train_x_final2 = pd.DataFrame(train_x_final2, columns = train_x_final.columns)
test_x_final2 = pd.DataFrame(test_x_final2, columns = test_x_final.columns)
df_child_x = pd.DataFrame(df_child_x, columns = test_x_final.columns)

### KNN Classifier

In [ ]:
# 성능 지표 (Accuracy, Precision, Recall, Specificity, F1-score)

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    
    print("Accuracy: ", round(Accuracy,3))
    print("Precision: ", round(Precision,3))
    print("Recall: ", round(Recall,3))
    print("Specificity: ", round(Specificity,3))

In [ ]:
# 혼동하지 않도록 변수명 통일
train_x = train_x_final2.copy()
train_y = train_y_final.copy()
test_x = test_x_final2.copy()
test_y = test_y_final.copy()

train_x = train_x.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)
test_x = test_x.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

train_x.shape, train_y.shape, test_x.shape, test_y.shape

In [ ]:
# KNeighborsClassifier + 그리드서치로 최적 하이퍼 파라미터 선정(기준: Recall)

knn_parameters = [{
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'leaf_size': [1, 3, 5, 10, 15, 20, 25, 30, 35, 40],
    'weights': ['uniform', 'distance']}]
# Stratified k-fold (default for classifier)
# n = 5 folds is default
knn_classifier = KNeighborsClassifier()
grid_cv = GridSearchCV(estimator = knn_classifier, param_grid = knn_parameters, cv=10, scoring='recall')
grid_cv.fit(train_x, train_y)

pred = grid_cv.predict(test_x)
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

In [ ]:
# 성능 확인
print(confusion_matrix(test_y, pred))
print(model_evaluation(test_y, pred))

### 모델 성능 평가

Accuracy는 0.888로 다소 높은 편이나, 본 과제의 목적은 사고 발생 위험지역을 선정하는 것이기 때문에, 다양한 성능 지표 중에서 **Recall** 에 집중하고자 한다.

**Recall 성능은 실제 사고 발생한 구역 중에서 모델이 사고 발생이라고 예측한 것의 비율을 나타내므로 본 주제와 가장 적합하다.**

KNN Classifier 모델의 Recall 성능은 0.352로 낮은 편에 속한다. 

### RFClassifier 모델링

In [ ]:
# RandomForest Classifier & 그리드 서치로 최적 하이퍼 파라미터 선정 (기준: Recall)
# 최적 파라미터 값 찾기
params = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, n_jobs = -1, scoring='recall')
grid_cv.fit(train_x, train_y)

pred = grid_cv.predict(test_x)
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

In [ ]:
# 성능 확인

print(confusion_matrix(test_y, pred))
print(model_evaluation(test_y, pred))

#### 모델 성능 평가

Recall 성능 측면에서는 0.63으로 Random Forest Classifier 모델은 KNN Classifier 모델보다 우수한 편에 속한다.


### lightgbm

In [ ]:
# lightgbm 버전 상, 한글 feature를 읽지 못하여 X1 ~ X10 변수로 처리
train_x.columns = ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']
train_y.columns = ['S']

test_x.columns = ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']
test_y.columns = ['S']

In [ ]:
# 모델링

start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 5, # 트리 깊이
             'num_leaves': 24,
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'metric': 'binary_logloss',
            'objective': 'binary', # 목적 함수
            'num_class': 1,
            'min_data_in_leaf': 400,
            'max_bin':1000} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
pred = np.round(lgb_model.predict(test_x),0) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, pred) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

In [ ]:
# 성능 확인

print(confusion_matrix(test_y, pred))
print(model_evaluation(test_y, pred))

### 모델 성능 평가

Recall 성능 측면에서 0.852로 다른 두 모델보다 우월한 수치이다. <br>
다시 말하면, **54개의 실제 사고가 일어난 격자 중 46개의 격자를 사고가 일어날 것이라고 예측하여 맞춘 우수한 모델**이다.

따라서, 본 과제에서는 구축한 LightGBM 모델을 통해 253개의 후보군 격자들의 사고 발생 확률을 도출하였다.

## 4. 후보군 격자들의 사고 발생 확률 예측

In [ ]:
# 격자의 중심좌표 불러오기 (경도 위도를 표현해주기 위해)
df2_1 = pd.DataFrame(columns=['격자명','경도','위도'])
for i in range(len(df2['features'])):
    a = df2['features'][i]['properties']['gid']
    c = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[0]
    d = np.mean(np.array(df2['features'][i]['geometry']['coordinates'][0][0])[:4,:],axis=0)[1]
    new_data = {'격자명':a, '경도': c,'위도': d}
    df2_1 = df2_1.append(new_data, ignore_index=True)

df2_1

In [ ]:
df1 = df2_1.iloc[list(df_child.index),:] #기존 df_child index를 통해 격자명을 불러온다
df1.reset_index(drop=True, inplace=True)

#모델을 통해 예측
s1 = pd.Series(lgb_model.predict(df_child_x))
df1['prob'] = s1

#예측 확률을 내림차순으로 정렬
df_end = df1.sort_values(by='prob',ascending=False)
df_end


#### 사고 발생 지역 상위 20 도출

In [ ]:
df_end.head(20)

#### 사고 발생 지역 하위 20 도출

In [ ]:
df_end.tail(20)

In [ ]:
# 283개의 평균 사고 발생 확률
df_end.prob.mean()

## 5. 어린이 교통사고 위험지역 20개소 선정

시각화를 통해 기존 어린이보호 구역의 시설명과 동일한 격자의 경우, 후보에서 제거하고 21번째 후보를 20번째 후보로 할당시킨다.

위의 작업을 반복하여 기존 어린이보호구역 시설명과 겹치지 않는 조건을 만족하는 위험격자 20개를 도출한다.

In [ ]:
#시각화를 위해 기존 격자의 MultiPolygon 데이터를 할당

df_MultiP = gpd.read_file('2.오산시_어린이교통사고_격자.geojson') # MutiPolygon 데이터를 가져오기위해 geopandas 를 이용
df_MultiP = df_MultiP[['gid','geometry']]
df_MultiP.rename(columns = {'gid':'격자명'}, inplace=True)

df_end = pd.merge(df_MultiP, df_end, on='격자명') #위험격자명과 MultiPolygon 데이터를 병합

df_end = df_end.sort_values(by='prob',ascending=False) # 사고 유발 확률로 내림차순 정렬

#후보군 교체를 위해 40개까지 할당
df_TOP40 = df_end.head(40)
df_TOP40.reset_index(drop=True,inplace=True) #인덱스별로 시각화하기 위해 초기화

In [ ]:
# 시각화 분석
import folium

m = folium.Map(
    location=[37.1516633,127.0359803], # 지도의 정 중앙지점을 표시
    zoom_start=12 # 초기 줌의 정도 설정
)
# df31은 오산시 경계
style2 = {'fillColor': 'white'}
folium.GeoJson( 
    df31,  
    name='child_pro',
    style_function=lambda x:style2,
).add_to(m) #행정동 경계를 지도내에 add

def GeoJson_where(json):
    style = {'fillColor': 'Red'}
    folium.GeoJson( 
    json,
    tooltip=f'{i}번째 위험구역',
   style_function=lambda x:style,
    name='child_pro').add_to(m) #행정동 경계를 지도 내에 add
    
# 상위 20개 할당

for i in range(0,20): #0번 ~ 19번
    try:
        GeoJson_where(df_TOP40['geometry'][i])
    except:
        print(i)


for i in range(len(df9)): # 기존 어린이 보호구역을 지도내에 마커로 add
    folium.Marker([df9.iloc[i]['보호구역_위도'], df9.iloc[i]['보호구역_경도']],
                  tooltip =df9.iloc[i]['시설명'],
                  icon = folium.Icon(color='blue',icon='home')).add_to(m)

In [ ]:
m

### 기존 어린이보호구역의 시설 위치와 동일한 격자 삭제

1번, 2번, 8번, 9번, 11번, 15번, 18번 위험지역 삭제 후, 상위 위험지역 7개 추가


In [ ]:
# 7개 지역 삭제
df_TOP40 = df_TOP40.drop([1,2, 8, 9, 11, 15, 18],axis=0)

# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803], # 지도의 정 중앙지점을 표시
    zoom_start=12 # 초기 줌의 정도 설정
)
# df31은 오산시 경계
style2 = {'fillColor': 'white'}
folium.GeoJson( 
    df31,  
    name='child_pro',
    style_function=lambda x:style2,
).add_to(m) #행정동 경계를 지도내에 add

for i in range(0,27): #0번 ~ 27번
    try:
        GeoJson_where(df_TOP40['geometry'][i])
    except:
        print(i,"번째 격자는 제거되었습니다")

for i in range(len(df9)): #어린이 보호구역을 지도내에 마커로 add
    folium.Marker([df9.iloc[i]['보호구역_위도'], df9.iloc[i]['보호구역_경도']],
              tooltip =df9.iloc[i]['시설명'],
              icon = folium.Icon(color='blue',icon='home')).add_to(m)

In [ ]:
m

## 재검토 후, 기존 어린이보호구역 시설 위치와 동일한 격자 삭제

26번 위험구역 삭제 후, 상위 1개 격자 추가

In [ ]:
# 1개 지역 삭제
df_TOP40 = df_TOP40.drop([26],axis=0)

# 시각화
m = folium.Map(
    location=[37.1516633,127.0359803], # 지도의 정 중앙지점을 표시
    zoom_start=12 # 초기 줌의 정도 설정
)
# df31은 오산시 경계
style2 = {'fillColor': 'white'}
folium.GeoJson( 
    df31,  
    name='child_pro',
    style_function=lambda x:style2,
).add_to(m) #행정동 경계를 지도내에 add

for i in range(0,28): #0번 ~ 27번
    try:
        GeoJson_where(df_TOP40['geometry'][i])
    except:
        print(i,"번째 격자는 제거되었습니다")

for i in range(len(df9)): #어린이 보호구역을 지도내에 마커로 add
    folium.Marker([df9.iloc[i]['보호구역_위도'], df9.iloc[i]['보호구역_경도']],
              tooltip =df9.iloc[i]['시설명'],
              icon = folium.Icon(color='blue',icon='home')).add_to(m)

# 최종 결과



In [ ]:
m.save('과제1_오산시위험지역20개.html')
m

In [ ]:
df_end = df_TOP40[:20]
df_end

In [ ]:
df_end.to_csv('오산시_어린이교통사고_위험지역20개소.csv',encoding='CP949')